In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.7.1+cu128
Is CUDA available: True


In [3]:
EPOCHS = 30
BATCH_SIZE = 64

In [4]:
transform = transforms.Compose([
transforms.ToTensor()
])
trainset = datasets.FashionMNIST(
    root = './.data/', 
    train = True,
    download = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root = './.data/', 
    train = False,
    download = True,
    transform = transform
)
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)
test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) # 학습 데이터를 DEVICE의 메모리로 보냄
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))


[1] Test Loss: 0.8434, Accuracy: 68.29%
[2] Test Loss: 0.6600, Accuracy: 76.83%
[3] Test Loss: 0.5863, Accuracy: 79.47%
[4] Test Loss: 0.5425, Accuracy: 80.91%
[5] Test Loss: 0.5524, Accuracy: 79.82%
[6] Test Loss: 0.5003, Accuracy: 82.46%
[7] Test Loss: 0.4999, Accuracy: 82.26%
[8] Test Loss: 0.4695, Accuracy: 83.32%
[9] Test Loss: 0.4753, Accuracy: 83.44%
[10] Test Loss: 0.4604, Accuracy: 83.67%
[11] Test Loss: 0.4650, Accuracy: 83.55%
[12] Test Loss: 0.4620, Accuracy: 83.65%
[13] Test Loss: 0.4312, Accuracy: 84.72%
[14] Test Loss: 0.4360, Accuracy: 84.34%
[15] Test Loss: 0.4337, Accuracy: 84.93%
[16] Test Loss: 0.4236, Accuracy: 85.20%
[17] Test Loss: 0.4114, Accuracy: 85.37%
[18] Test Loss: 0.4041, Accuracy: 85.67%
[19] Test Loss: 0.4178, Accuracy: 85.41%
[20] Test Loss: 0.4235, Accuracy: 84.63%
[21] Test Loss: 0.3949, Accuracy: 85.96%
[22] Test Loss: 0.3939, Accuracy: 86.20%
[23] Test Loss: 0.3980, Accuracy: 85.93%
[24] Test Loss: 0.3996, Accuracy: 85.95%
[25] Test Loss: 0.3964, A

## Overfitting

In [11]:
train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./.data', 
            train=True, download=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])), batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./.data', train=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,)) ])), batch_size=BATCH_SIZE, shuffle=True)

## Dropout

In [16]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p # 드롭아웃 확률
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = F.relu(self.fc2(x))
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x)
        return x


In [17]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)